# Procesamiento de Lenguaje Natural con LLMs
### Modelo Clásico a Modelos Modernos 🚀

Este notebook muestra la evolución del NLP:
1. Fundamentos clásicos (preprocesamiento y representación)
2. Clasificación tradicional
3. LLMs modernos

## PARTE 1: FUNDAMENTOS CLÁSICOS DE NLP

In [ ]:
!pip install nltk transformers torch scikit-learn -q

In [ ]:
import nltk
import re
from collections import Counter
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

In [ ]:
# Dataset simple para clasificación de sentimientos
reviews = [
    "Me encanta este producto, es increíble!",
    "Pésima experiencia, no lo recomiendo",
    "Es aceptable, cumple lo esperado",
    "Excelente compra, superó expectativas",
    "Horrible servicio, muy decepcionado"
]
sentimientos = ['positivo', 'negativo', 'neutral', 'positivo', 'negativo']

print("=== DATASET ORIGINAL ===")
for i, (review, sent) in enumerate(zip(reviews, sentimientos), 1):
    print(f"{i}. [{sent}] {review}")

### Preprocesamiento Clásico

**PREPROCESAMIENTO**: Limpiar y normalizar texto para análisis.
- Tokenización: dividir en palabras
- Stop words: remover palabras comunes sin significado
- Stemming: reducir palabras a su raíz

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

def preprocesar(texto):
    # Limpiar y tokenizar
    texto = re.sub(r'[^\w\s]', '', texto.lower())
    tokens = word_tokenize(texto)
    # Remover stop words y hacer stemming
    tokens = [stemmer.stem(t) for t in tokens if t not in stop_words]
    return ' '.join(tokens)

reviews_procesados = [preprocesar(r) for r in reviews]

print("\n=== DESPUÉS DEL PREPROCESAMIENTO ===")
for i, review in enumerate(reviews_procesados, 1):
    print(f"{i}. {review}")

### Representación: Bag of Words

**BAG OF WORDS**: Representa texto como vector de frecuencias de palabras.
Cada palabra del vocabulario es una dimensión.
Problema: Pierde contexto y semántica.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_procesados)

print("\n=== BAG OF WORDS (TF-IDF) ===")
print(f"Vocabulario: {vectorizer.get_feature_names_out()}")
print(f"Dimensión: {X.shape}")

### Clasificación Tradicional

**CLASIFICACIÓN CLÁSICA**: Usamos algoritmos como Naive Bayes o SVM.
Requieren mucho preprocesamiento y features engineering.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

clf = MultinomialNB()
scores = cross_val_score(clf, X, sentimientos, cv=2)
print(f"\n=== CLASIFICACIÓN TRADICIONAL (Naive Bayes) ===")
print(f"Accuracy promedio: {scores.mean():.2f}")

## PARTE 2: ¡LA REVOLUCIÓN DE LOS LLMs! 🤖✨

**LLMs (Large Language Models)**: Modelos masivos entrenados en billones de palabras.
- Entienden contexto, semántica y relaciones complejas
- No necesitan preprocesamiento manual
- Pueden hacer múltiples tareas sin reentrenamiento (zero-shot)
- Ejemplos: BERT, GPT, LLaMA, etc.


In [ ]:
from transformers import pipeline

print("\n" + "="*70)
print("🚀 CARGANDO MODELO DE LENGUAJE MODERNO...")
print("="*70)

# Cargamos un modelo preentrenado en español para análisis de sentimientos
# Este modelo ya fue entrenado en millones de textos
clasificador_llm = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis"
)

print("✅ Modelo cargado: RoBERTuito (BERT en español)")
print("   Entrenado en millones de tweets en español\n")

### Análisis con LLM - Zero Shot

**ZERO-SHOT**: El modelo puede clasificar SIN ver ejemplos de entrenamiento.
Ya aprendió sobre sentimientos de su entrenamiento masivo.

In [ ]:
print("=== CLASIFICACIÓN CON LLM (Sin preprocesamiento) ===\n")

for i, review in enumerate(reviews, 1):
    resultado = clasificador_llm(review)[0]
    etiqueta = resultado['label']
    confianza = resultado['score']
    
    # Mapeo de etiquetas
    etiqueta_es = {'POS': '😊 Positivo', 'NEG': '😞 Negativo', 'NEU': '😐 Neutral'}
    
    print(f"{i}. \"{review}\"")
    print(f"   Predicción: {etiqueta_es.get(etiqueta, etiqueta)} (confianza: {confianza:.2%})")
    print()

### Casos más complejos

In [ ]:
print("\n" + "="*70)
print("🧪 PROBANDO CON CASOS DESAFIANTES")
print("="*70 + "\n")

casos_complejos = [
    "No es que sea malo, pero definitivamente no es bueno",  # Negación compleja
    "Esperaba más, aunque tiene sus puntos positivos",       # Sentimiento mixto
    "¡Qué desastre tan espectacular! (Es ironía)",          # Ironía
    "Increíble cómo algo tan caro puede ser tan mediocre"   # Sarcasmo
]

for caso in casos_complejos:
    resultado = clasificador_llm(caso)[0]
    etiqueta = {'POS': '😊', 'NEG': '😞', 'NEU': '😐'}.get(resultado['label'], '🤔')
    
    print(f"📝 \"{caso}\"")
    print(f"   {etiqueta} {resultado['label']} ({resultado['score']:.1%})\n")

## PARTE 3: GENERACIÓN DE TEXTO CON LLMs 🎨

Los LLMs también pueden **GENERAR** texto coherente.
Vamos a usar un modelo generativo en español.

In [ ]:
print("\n" + "="*70)
print("🎨 GENERACIÓN DE TEXTO")
print("="*70 + "\n")

# Modelo generativo (más pequeño para Google Colab)
generador = pipeline(
    "text-generation",
    model="DeepESP/gpt2-spanish",
    max_length=80
)

prompts = [
    "La inteligencia artificial es",
    "En el futuro, los robots",
    "El curso de IA me parece"
]

for prompt in prompts:
    texto_generado = generador(prompt, do_sample=True, temperature=0.7)[0]['generated_text']
    print(f"💭 Prompt: \"{prompt}\"")
    print(f"✨ Generado: {texto_generado}\n")

## PARTE 4: COMPARACIÓN Y CONCLUSIONES

In [ ]:
print("\n" + "="*70)
print("📊 MÉTODOS CLÁSICOS vs LLMs")
print("="*70 + "\n")

comparacion = """
┌─────────────────────────┬──────────────────────┬─────────────────────┐
│ ASPECTO                 │ MÉTODOS CLÁSICOS     │ LLMs                │
├─────────────────────────┼──────────────────────┼─────────────────────┤
│ Preprocesamiento        │ Extensivo y manual   │ Mínimo o ninguno    │
│ Features                │ Manual (BoW, TF-IDF) │ Automáticas         │
│ Contexto                │ Limitado             │ Excelente           │
│ Semántica               │ Básica               │ Profunda            │
│ Datos necesarios        │ Miles por clase      │ Zero/Few-shot       │
│ Velocidad inferencia    │ Muy rápida           │ Más lenta           │
│ Recursos computación    │ Bajos                │ Altos               │
│ Tareas múltiples        │ Un modelo por tarea  │ Modelo multiuso     │
└─────────────────────────┴──────────────────────┴─────────────────────┘
"""

print(comparacion)

## BONUS: Clasificación Personalizada con Few-Shot

**FEW-SHOT LEARNING**: Damos al LLM solo unos pocos ejemplos y aprende la tarea.
¡No necesita reentrenamiento completo!

In [ ]:
print("\n" + "="*70)
print("🎯 BONUS: FEW-SHOT LEARNING")
print("="*70 + "\n")

# Usamos un modelo más potente para few-shot
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

modelo_nombre = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
modelo = AutoModelForSequenceClassification.from_pretrained(
    modelo_nombre,
    num_labels=3
)

print("💡 Con few-shot, el modelo aprende de pocos ejemplos y generaliza.")
print("   Útil cuando tienes datos limitados o tareas muy específicas.\n")

## CONCLUSIONES Y RECURSOS

In [ ]:
print("\n" + "="*70)
print("🎓 CONCLUSIONES")
print("="*70 + "\n")

print("""
🔗 RECURSOS:
   • HuggingFace Models: https://huggingface.co/models
   • Transformers Library: https://huggingface.co/docs/transformers
   • Papers With Code (NLP): https://paperswithcode.com/area/natural-language-processing
""")

print("\n✨ ¡El futuro del NLP es emocionante! Sigue explorando... ✨\n")